In [ ]:
import math

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix

In [ ]:
byAge={
    '2-11':,
    '12-17':
    '18-34':
    '35-49':
    '50-64':
    '65+':
}
byIncome={
   '<25k':
   '25-50k':
   '50-75k':
   '>75k':
}
class people:
    def __init__(self, age, income):
      self.age=age
      self.income=income
    